In [1]:
import chardet
from BeautifulSoup import BeautifulSoup 
import re
import os
from nltk.stem.porter import PorterStemmer
from langdetect import detect
from langdetect import detect_langs
import pymorphy2
import regex
import cPickle as pickle
import codecs
import docx
import numpy as np
import pandas as pd
import html2text
import ipyparallel as ipp
import gensim
from gensim import corpora
from gensim import models
import operator
from gensim import similarities
import sys
from nltk.corpus import stopwords
from gensim.summarization.bm25 import BM25
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# ipcluster nbextension enable

def bprint(l):
    for w in l:
        print w
        
CNT_TRAIN_DOCS = 394955
CNT_QUERY = 4125
CNT_CLICK_INFO = 319986
#582 недодокумента

In [59]:
with open("Spisok_gorodov_Rossii.txt", "r") as f:
    city_list = []
    for line in f:
        city_list.append(line.decode("windows-1251").strip().lower())
    city_list = list(set(city_list))

In [60]:
print len(city_list), bprint(city_list[:10])

1073 черкесск
иланский
олонец
златоуст
мосальск
малоярославец
бирюсинск
подпорожье
дигора
черняховск
None


In [130]:
stop_words = stopwords.words('russian')
# bprint(stop_words)

In [5]:
letters_dict = {'q':u'й', 'w':u'ц', 'e':u'у', 'r':u'к', 't':u'е', 'y':u'н', \
                'u':u'г', 'i':u'ш', 'o':u'щ', 'p':u'з', '[':u'х', ']':u'ъ', \
                'a':u'ф', 's':u'ы', 'd':u'в', 'f':u'а', 'g':u'п', 'h':u'р', \
                'j':u'о', 'k':u'л', 'l':u'д', ';':u'ж', '\'':u'э', 'z':u'я', \
                'x':u'ч', 'c':u'с', 'v':u'м', 'b':u'и', 'n':u'т', 'm':u'ь',\
                ',':u'б', '.':u'ю', '`':u'ё', '?':u',', u' ':u' ', u'е':u'e', \
                u'й':u'й', u'ц':u'ц', u'у':u'у', u'к':u'к', u'н':u'н', u'г':u'г', \
                u'ш':u'ш', u'щ':u'щ', u'з':u'з', u'х':u'х', u'ъ':u'ъ', u'ф':u'ф', \
                u'ы':u'ы', u'в':u'в', u'а':u'а', u'п':u'п', u'р':u'р', u'о':u'о', \
                u'л':u'л', u'д':u'д', u'ж':u'ж', u'э':u'э', u'я':u'я', u'ч':u'ч', \
                u'с':u'с', u'м':u'м', u'и':u'и', u'т':u'т', u'ь':u'ь', u'б':u'б', \
                u'ю':u'ю',  u'ё':u'ё', u'\u0456':u'\u0456'}

def inverse_letters(tokens):
    new_tokens = []
    for token in tokens:
        new_token = ""
        for letter in token:
            if letter in letters_dict:
                new_token += letters_dict[letter]
            else:
                new_token += letter
        new_tokens.append(new_token)
    
    return new_tokens

In [45]:
#обработка запросов - конвертация неправильной раскладки
q_file = open("queries.tsv", "r")
new_q_file = open("queries_true.tsv", "w")
for idx, q_line in enumerate(q_file):
    q_idx = q_line.strip().split("\t")[0]
    query = q_line.strip().split("\t")[1].decode("utf-8")
    lang = detect(query)

    true_lang = ['uk', 'ru', 'en']
    if lang not in true_lang:
        print "CHANGE \t", " ".join(inverse_letters(query.split(" ")))
        new_q_file.write((q_idx + "\t" + \
                         " ".join(inverse_letters(query.split(" "))) + "\n").encode("utf-8"))
    else:
        print q_line.strip().split("\t")[1], detect_langs(query)
        new_q_file.write((q_idx + "\t" + query + "\n").encode("utf-8"))
        
q_file.close()
new_q_file.close()

1 как называется форма приведенная на рисунке фио какую организацию представляет должность дата и способ направления приглашения подтверждение получения приглашения дата согласие принять участие в работе да нет предполагает ли выступить тема выступления нуждается ли в гостинице нуждается ли в билетах на обратный путь самолет поезд и т п прибыл на заседание 1 2 3 4 5 6 7 8 9 киселева мария владимировна ао восток главный бухгалтер 12 04 14 письмо приглашение 14 04 14 да нет нет нет да [ru:0.999997667749]
1 месяц после операции на кишечнику диета что можно добавить [ru:0.999996265307]
1 положительный и 1 отрицательный могут ли [ru:0.999995788976]
10 книг которые нужно прочитать что бы быть спокойным [ru:0.99999900847]
100 к 1 что нужно взять в [ru:0.999997252116]
2 правды 1 ложь что можно придумать [ru:0.999996922135]
2016 действует ли зао рождественская мануфактура [ru:0.999996186897]
31 можно ли изменять оформление списков по своему усмотрению [ru:0.999996620688]
CHANGE 	самыи лучши ком

In [13]:
detect_langs("rfr")

[cy:0.999995053614]

In [2]:
Porter = PorterStemmer()
RusLem = pymorphy2.MorphAnalyzer()
UkLem = pymorphy2.MorphAnalyzer(lang='uk')
lang = {}
words_cache = {}

def get_document(document_id):
    if int(document_id) == 0:
        file_name = "./docs/docs_norm_1.tsv"
    else:
        file_name = "./docs/docs_norm_" + str((int(document_id) - 1) / 1000 + 1) + ".tsv"
    docs_file = open(file_name, "r")
    ok = 0
    for doc_line in docs_file:
        if doc_line.split("\t")[0] == document_id:
            return doc_line
            ok = 1

    if ok != 1:
        print "doc not found"
    docs_file.close()


def normalize_text(text, language):
    #цифры стоит оставлять
    tokens = re.findall('[\w]+',text.lower(), re.U)
    #теперь стемминг зависит от языка. Надо его определить.
    if language == u'ru':
#         print "rus"
        if 'ru' in lang:
            lang['ru'] += 1
        else:
            lang['ru'] = 1
        
#         tokens = [RusLem.parse(token)[0].normal_form for token in tokens]
        tokens_res = []
        for token in tokens:
            if token.isdigit():
                tokens_res.append(token)
            elif token in words_cache:
                tokens_res.append(words_cache[token])
            else:
                word = RusLem.parse(token)[0].normal_form
                tokens_res.append(word)
                try:
                    words_cache[token] = word
                except:
                    pass
                
                
    elif language == u'uk':
        if 'uk' in lang:
            lang["uk"] += 1
        else:
            lang["uk"] = 1
        
#         tokens = [UkLem.parse(token)[0].normal_form for token in tokens]
        tokens_res = []
        for token in tokens:
            if token.isdigit():
                tokens_res.append(token)
            elif token in words_cache:
                tokens_res.append(words_cache[token])
            else:
                word = UkLem.parse(token)[0].normal_form
                tokens_res.append(word)
                try:
                    words_cache[token] = word
                except:
                    pass
                
    elif language == u'en':
        if 'en' in lang:
            lang["en"] += 1
        else:
            lang["en"] = 1
            
        tokens_res = []
        for token in tokens:
            if token.isdigit():
                tokens_res.append(token)
            elif token in words_cache:
                tokens_res.append(words_cache[token])
            else:
                word = Porter.stem(token)
                tokens_res.append(word)
                try:
                    words_cache[token] = word
                except:
                    pass
    else:
#         print "can not prepare language:", language
        return " ".join(tokens)
    return " ".join(tokens_res)

In [3]:
#нормализовываем наши дофига большие тренировочные данные
#складываем в файлики по 1000 документов
def get_nums_ready_docs(file_name):
    docs_file = open(file_name, "r")
    already_prepare_file = []
    for idx, line in enumerate(docs_file):
        line = line.strip().split("\t")
        already_prepare_file.append(line[0])
#         sys.stderr.write("\rPrepare %d / %d line" % \
#                      (idx, 1000))
#     print len(already_prepare_file)
    docs_file.close()
    return already_prepare_file

norm_docs_file.close()
docs_file = open("docs.tsv", "r")
# norm_docs_file = open("./docs/docs_norm_1.tsv", "w")
already_prepare_file = get_nums_ready_docs("./docs/docs_norm_1.tsv")
norm_docs_file = open("./docs/docs_norm_1.tsv", "a")
cur_file = 1
not_prepare = 2383
for idx, doc_line in enumerate(docs_file):
    if idx < (cur_file - 1) * 1000:
        continue
    if idx > cur_file * 1000:
        norm_docs_file.close()
        cur_file += 1
        already_prepare_file = get_nums_ready_docs("./docs/docs_norm_" + str(cur_file) + ".tsv")
        norm_docs_file = open("./docs/docs_norm_" + str(cur_file) + ".tsv", "a")
#     if idx > 0:
#         break
        
    line = doc_line.strip().split("\t")
    doc_id = line[0]
    if doc_id not in already_prepare_file:
        if len(line) == 1:
            norm_docs_file.write((doc_id + "\n").encode("utf-8"))
        elif len(line) == 2:
            doc_title = line[1].decode("utf-8")
            try:
                language = detect(doc_text)
            except Exception as e:
                print e
                not_prepare += 1
                language = 'ru'
            if language not in ['ru', 'en', 'uk']:
                print "can not prepare language:", language
                not_prepare += 1
                language = 'ru'
            norm_docs_file.write((doc_id + "\t" + normalize_text(doc_title, language) + "\n").encode("utf-8"))
        elif len(line) == 3:
            doc_title = line[1].decode("utf-8")
            doc_text = line[2].decode("utf-8")
            try:
                language = detect(doc_text)
            except Exception as e:
                print e
                not_prepare += 1
                language = 'ru'
            if language not in ['ru', 'en', 'uk']:
                print "can not prepare language:", language
                not_prepare += 1
                language = 'ru'
            norm_docs_file.write((doc_id + "\t" + normalize_text(doc_title, language) + "\t" + \
                                  normalize_text(doc_text, language) + "\n").encode("utf-8"))
        else:
            print "Content > 3"

    sys.stderr.write("\rPrepare %d / %d documents, content: %d, words in cache: %d, not prepare: %d" % \
                     (idx, CNT_TRAIN_DOCS, len(line), len(words_cache), not_prepare))
    
docs_file.close()
norm_docs_file.close()

NameError: name 'norm_docs_file' is not defined

In [96]:
docs_file = open("docs.tsv", "r")
cur_file = 1
for idx, doc_line in enumerate(docs_file):
        
    line = doc_line.strip().split("\t")
    doc_id = line[0]
    if doc_id == "734":
        print doc_line
    sys.stderr.write("\rPrepare %d / %d documents" % \
                     (idx, CNT_TRAIN_DOCS))
docs_file.close()

Prepare 1514 / 394955 documents

734	AVIANEWS АГРО НОВИНИ	ЕСЛИ ВЫ НИ РАЗУ НЕ ИГРАЛИ В СЛОТЫ ОНЛАЙН НЕ СТОИТ ОТЧАИВАТЬСЯ ВАМ НУЖНО ПОСЕТИТЬ АЗАРТНЫЙ КЛУБ 777AVTOMATYVULKAN COM ЗДЕСЬ СОЗДАНЫ ВСЕ УСЛОВИЯ ДЛЯ УСПЕШНОЙ ИГРЫ ВЫ ДУМАЕТЕ ЧТО НУЖНО ОБЛАДАТЬ ОПРЕДЕЛЕННЫМИ НАВЫКАМИ ДЛЯ ТОГО ЧТОБЫ ИГРАТЬ В АВТОМАТЫ ЭТО В ДЕЙСТВИТЕЛЬНОСТИ НЕ ТАК ЗАЙДИТЕ НА САЙТ КАЗИНО ВУЛКАН НАЧИНИТЕ ИГРУ И ВЫ САМИ ПОЙМЕТЕ ЧТО ВСЕ ОЧЕНЬ ПРОСТО ЗДЕСЬ ВАМИ БУДУТ НАЙДЕНЫ СИМУЛЯТОРЫ ОТ ТАКИХ ГИГАНТОВ КАК МУЛЬТИ ГАМИНАТОР ИГРОСОФТ МЕГА ДЖЕК ПРОДОВЖУВАТИ ЧИТАННЯ ВСЕ УСЛОВИЯ ДЛЯ УСПЕШНОЙ ИГРЫ ЕСЛИ ВЫ НИ РАЗУ НЕ ИГРАЛИ В СЛОТЫ ОНЛАЙН НЕ СТОИТ ОТЧАИВАТЬСЯ ВАМ НУЖНО ПОСЕТИТЬ АЗАРТНЫЙ КЛУБ 777AVTOMATYVULKAN COM ЗДЕСЬ СОЗДАНЫ ВСЕ УСЛОВИЯ ДЛЯ УСПЕШНОЙ ИГРЫ ВЫ ДУМАЕТЕ ЧТО НУЖНО ОБЛАДАТЬ ОПРЕДЕЛЕННЫМИ НАВЫКАМИ ДЛЯ ТОГО ЧТОБЫ ИГРАТЬ В АВТОМАТЫ ЭТО В ДЕЙСТВИТЕЛЬНОСТИ НЕ ТАК ЗАЙДИТЕ НА САЙТ КАЗИНО ВУЛКАН НАЧИНИТЕ ИГРУ И ВЫ САМИ ПОЙМЕТЕ ЧТО ВСЕ ОЧЕНЬ ПРОСТО ЗДЕСЬ ВАМИ БУДУТ НАЙДЕНЫ СИМУЛЯТОРЫ ОТ ТАКИХ ГИГАНТОВ КАК МУЛЬТИ ГАМИНАТОР ИГРОСОФТ МЕГА ДЖЕК ПРОДОВЖУВАТИ ЧИТ

Prepare 10536 / 394955 documents

KeyboardInterrupt: 

In [55]:
#нормализовываем запросы
query_file = open("queries_true.tsv", "r")
norm_query_file = open("./queries_norm.tsv", "w")

for idx, query_line in enumerate(query_file):
    line = query_line.strip().split("\t")
    query_id = line[0]
    query = line[1].decode("utf-8")
    try:
        language = detect(query)
    except Exception as e:
        print e
        continue
    if language not in ['ru', 'en', 'uk']:
        print "can not prepare language:", language
        language = u'ru'
    
    norm_query_file.write((query_id + "\t" + normalize_text(query, language) + "\n").encode("utf-8"))
        
    sys.stderr.write("\rPrepare %d / %d query" % \
                     (idx, CNT_QUERY))
    

query_file.close()
norm_query_file.close()

Prepare 48 / 4125 query

can not prepare language: mk
can not prepare language: bg
can not prepare language: bg
can not prepare language: bg
can not prepare language: mk


Prepare 69 / 4125 query

can not prepare language: mk
can not prepare language: bg
can not prepare language: mk


Prepare 113 / 4125 query

...

Prepare 3996 / 4125 query

can not prepare language: mk


Prepare 4069 / 4125 query

can not prepare language: mk


Prepare 4124 / 4125 query

In [105]:
#скомбинируем вместе документы по каждому запросу - отдельно текст и отдельно заголовок
test_query_file = open("test.tsv", "r")
cur_query = 233
for idx, query_line in enumerate(test_query_file):
    if idx < cur_query:
        continue
    query_id = query_line.split("\t")[0]
    doc_nums = query_line.strip().split("\t")[1][1:-1].split(",")
    query_title_file = open("./query_docs/title_"+ query_id, "w")
    query_text_file = open("./query_docs/text_"+ query_id, "w")
    
    for idx_doc, doc in enumerate(doc_nums):
        #высчитываем название файла, в котором хранится документ с номером doc
        if doc == "0":
            file_name = "./docs/docs_norm_1.tsv"
        else:
            file_name = "./docs/docs_norm_" + str((int(doc) - 1) / 1000 + 1) + ".tsv"
        docs_file = open(file_name, "r")
        ok = 0
        for doc_line in docs_file:
            if doc_line.strip().split("\t")[0] == doc:
                if len(doc_line.strip().split("\t")) == 1:
                    query_title_file.write(doc_line)
                    query_text_file.write(doc_line)
                elif len(doc_line.strip().split("\t")) == 2:
                    query_title_file.write("\t".join(doc_line.strip().split("\t")[0:2]) + "\n")
                    query_text_file.write(doc_line.split("\t")[0] + "\n")
                else:
                    query_title_file.write("\t".join(doc_line.strip().split("\t")[0:2]) + "\n")
                    query_text_file.write("\t".join([doc_line.split("\t")[0], doc_line.split("\t")[2]]))
                ok = 1
                
        if ok != 1:
            print "doc not found", doc
        docs_file.close()
        
        sys.stderr.write("\rPrepare %d / %d query; Document: %d / %d" % \
                     (idx, CNT_QUERY, idx_doc, len(doc_nums)))
        
    query_file.close()

test_query_file.close()

Prepare 815 / 4125 query; Document: 106 / 107

In [348]:
#скомбинируем вместе документы по каждому запросу - отдельно текст и отдельно заголовок
test_query_file = open("clicks.train.tsv", "r")
cur_query = 814
for idx, query_line in enumerate(test_query_file):
    if not (820 > idx > 810):
        continue
    query_id = query_line.split("\t")[0]
    doc_nums = query_line.strip().split("\t")[1][1:-1].split(",")
    query_title_file = open("./query_docs/title_"+ query_id, "w")
    query_text_file = open("./query_docs/text_"+ query_id, "w")
    
    for idx_doc, doc in enumerate(doc_nums):
        #высчитываем название файла, в котором хранится документ с номером doc
        if doc == "0":
            file_name = "./docs/docs_norm_1.tsv"
        else:
            file_name = "./docs/docs_norm_" + str((int(doc) - 1) / 1000 + 1) + ".tsv"
        docs_file = open(file_name, "r")
        ok = 0
        for doc_line in docs_file:
            if doc_line.strip().split("\t")[0] == doc:
                if len(doc_line.strip().split("\t")) == 1:
                    query_title_file.write(doc_line)
                    query_text_file.write(doc_line)
                elif len(doc_line.strip().split("\t")) == 2:
                    query_title_file.write("\t".join(doc_line.strip().split("\t")[0:2]) + "\n")
                    query_text_file.write(doc_line.split("\t")[0] + "\n")
                else:
                    query_title_file.write("\t".join(doc_line.strip().split("\t")[0:2]) + "\n")
                    query_text_file.write("\t".join([doc_line.split("\t")[0], doc_line.split("\t")[2]]))
                ok = 1
                
        if ok != 1:
            print "doc not found", doc
        docs_file.close()
        
        sys.stderr.write("\rPrepare %d / %d query; Document: %d / %d" % \
                     (idx, CNT_QUERY, idx_doc, len(doc_nums)))
        
    query_file.close()

test_query_file.close()

Prepare 819 / 4125 query; Document: 104 / 105

In [72]:
print get_document("27320")

doc not found
None


In [73]:
bprint (words_cache.keys()[1000:1010])

качественное
тихоновна
тихоновны
deya
xtrmnatedabumz
тибета
яркими
molchilar
участии
tartibga


In [94]:
"123\n".split("\t")[0]

'123\n'

In [74]:
print len(words_cache)

31758


In [98]:
!cat ./docs/docs_norm_1.tsv | head -n 740 | tail -n 10

In [15]:
norm_docs_file = open("./docs/docs_norm_1.tsv", "r")
for idx, doc_line in enumerate(norm_docs_file):
#     if idx > 5:
#         break
    l = doc_line.strip().split("\t")
    doc_id = l[0]
    doc_title = l[1].decode("utf-8")
#     doc_text = l[2].decode("utf-8")
    print doc_id, doc_title

0 smartjs academy архитектура js приложение харьков
1 essay on importance of cleanliness at home school and neighbourhood online consultant
2 доска на два колесо как называться
3 my career view topic eboni teen upskirt
4 embassy of the republic of kazakhstan to the united arab emirates 14
5 скачать promt expert 10 build 9 0 526 беспалтный офис
6 press releases embassy of the republic of uzbekistan
7 как добраться международный аэропорт казань
8 как добраться на машина до аэропорт шереметьево
9 создание лендинг в спб от 34900 гарантия в договор разработка посадочный страница под ключ
10 код активация для касперский 9 на андроид софт 99
11 гей онлайн чат знакомство дима питров вконтакте на chiquymei id ai
12 запись вирт в чат знакомство онлайн видео чат с хороший девушка рунет на chiquymei id ai
13 знакомство общение чат в г барнаул знакомство г на chiquymei id ai
14 знакомство общение веб чат чат icq skype миранда интернет запорожье 5 сайт на chiquymei id ai
15 болеть за футбол и трахат

In [10]:
for f in sorted(os.listdir("./docs/")):
    t = "./docs/" + f
#     t = "./docs/docs_norm_10.tsv" 
    print t
    !cat $t | head -c 8
    !cat $t | tail -n 1 | head -c 8
    print

./docs/docs_norm_1.tsv
0	smartjcat: ошибка записи: Обрыв канала
1000	б�tail: ошибка записи: Обрыв канала

./docs/docs_norm_10.tsv
9001	и�cat: ошибка записи: Обрыв канала
10000		�tail: ошибка записи: Обрыв канала

./docs/docs_norm_100.tsv
99001	гcat: ошибка записи: Обрыв канала
100000	�tail: ошибка записи: Обрыв канала

./docs/docs_norm_101.tsv
100001	�cat: ошибка записи: Обрыв канала
101000	�tail: ошибка записи 'standard output': Обрыв канала

./docs/docs_norm_102.tsv
101001	�cat: ошибка записи: Обрыв канала
102000	�tail: ошибка записи: Обрыв канала

./docs/docs_norm_103.tsv
102001	�cat: ошибка записи: Обрыв канала
103000	�tail: ошибка записи 'standard output': Обрыв канала

./docs/docs_norm_104.tsv
103001	�cat: ошибка записи: Обрыв канала
104000	�tail: ошибка записи: Обрыв канала

./docs/docs_norm_105.tsv
104001	�cat: ошибка записи: Обрыв канала
^C

./docs/docs_norm_106.tsv
105001	�cat: ошибка записи: Обрыв канала
106000	ntail: ошибка записи: Обрыв канала

./docs/docs_norm_107.tsv
106

In [11]:
f = open("test_query_norm", "w")
for t in sorted(test_data.keys()):
    f.write(str(t) + "\t" + get_query(t).encode("utf-8") + "\n")
f.close()

In [35]:
docs_file = open("docs.tsv", "r")
cnt = 0
for idx, doc_line in enumerate(docs_file):
    l = doc_line.strip().split("\t")
    if len(l) != 3:
        print len(l), idx, doc_line
#         print l[1]
#         print l[2]
#         break
        cnt += 1
print cnt
docs_file.close()

2 1145 1145	ЭРОТИЧЕСКИЙ РЕЛАКС В СПБ	

2 1565 1565	YEVPATORIYA BIZ NBSPДАННЫЙ ВЕБ САЙТ ВЫСТАВЛЕН НА ПРОДАЖУ NBSPYEVPATORIYA РЕСУРСЫ И ИНФОРМАЦИЯ	

1 1913 1913		

1 3813 3813		

1 5011 5011		

1 5012 5012		

1 5016 5016		

1 5017 5017		

1 5018 5018		

1 5472 5472		

2 5646 5646	ГОЛОВНА	

1 6435 6435		

1 8186 8186		

1 8237 8237		

1 8238 8238		

2 8752 8752	OFFER	

2 8753 8753	CLOSED SHUT COM BY RAFAËL ROZENDAAL 2009	

1 8787 8787		

1 8883 8883		

1 8884 8884		

1 8885 8885		

1 8886 8886		

1 8887 8887		

1 8888 8888		

1 8889 8889		

1 9849 9849		

1 9866 9866		

2 10053 10053	DISQUS КАК ОБНОВИТЬ СИМС 4 ПИРАТКУ ДО ПОСЛЕДНЕЙ ВЕРСИИ	

1 10255 10255		

2 10783 10783	ЦЕНТР НЕДВИЖИМОСТИ ЕКАТЕРИНБУРГ СИТИ	

1 11306 11306		

1 11307 11307		

1 12001 12001		

1 12002 12002		

1 12657 12657		

2 12956 12956	ГДЕПОСЫЛКА ОТСЛЕЖИВАНИЕ ПОЧТОВЫХ ОТПРАВЛЕНИЙ ИНТЕРНЕТ МАГАЗИН CHROME	

2 12957 12957	ADBLOCK ИНТЕРНЕТ МАГАЗИН CHROME	

2 12958 12958	ADBLOCK PLUS ИНТЕРНЕТ МАГАЗИН CHROME	

2 12959 12959	

## Текстовое ранжирование

In [3]:
#строим gensim dictionary
def create_dict(docs_dir):
    files = os.listdir(docs_dir)
    dictionary = corpora.Dictionary(( (line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3  else "") ).split(" ")\
                                    for line in open(docs_dir + files[0]))
    sys.stderr.write("\rPrepare %d / %d documents, dictionary: %s" % \
                     (dictionary.num_docs, CNT_TRAIN_DOCS, str(dictionary)))
    for f in files:
        dictionary.add_documents(( (line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3  else "") ).split(" ")\
                                    for line in open(docs_dir + f))
        sys.stderr.write("\rPrepare %d / %d documents, dictionary: %s" % \
                     (dictionary.num_docs, CNT_TRAIN_DOCS, str(dictionary)))
        
    dictionary.save("./dictionary_gensim")
    
def get_bigram(line):
    l = ((line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3  else ""))\
                                    .split(" ")
    x = [l[i] + " " + l[i + 1] for i in range(len(l) - 1)]
    return x
    
#строим gensim dictionary для биграмм
def create_dict_2(docs_dir):
    files = os.listdir(docs_dir)
    dictionary = corpora.Dictionary(get_bigram(line) for line in open(docs_dir + files[0]))
    sys.stderr.write("\rPrepare %d / %d documents, dictionary: %s" % \
                     (dictionary.num_docs, CNT_TRAIN_DOCS, str(dictionary)))
    for f in files:

        dictionary.add_documents(get_bigram(line) for line in open(docs_dir + f))
        sys.stderr.write("\rPrepare %d / %d documents, dictionary: %s" % \
                     (dictionary.num_docs, CNT_TRAIN_DOCS, str(dictionary)))
        
    dictionary.save("./dictionary_2_gensim")

In [5]:
create_dict("./docs/")

NameError: name 'create_dict' is not defined

In [15]:
create_dict_2("./docs/")

Prepare 395955 / 394955 documents, dictionary: Dictionary(4521261 unique tokens: [u'\u0437\u043d\u0430\u0447\u0435\u043d\u0438\u0435 \u0432\u0438\u0434\u0435\u043e\u043d\u044f\u043d\u044f', u'\u0441\u0435\u0431\u044f \u0430\u043a\u0442\u0443\u0430\u043b\u044c\u043d\u044b\u0439', u'\u043f\u0430\u0432\u0435\u043b \u043f\u0438\u0441\u043a\u0430\u0440\u0435\u0432', u'canada goos', u'\u0440\u0430\u0437\u043c\u0430\u0445 \u0441\u043e\u043f\u043e\u0441\u0442\u0430\u0432\u0438\u043c\u044b\u0439']...)

In [4]:
dictionary = corpora.Dictionary.load("./dictionary_gensim")

In [5]:
dictionary_2 = corpora.Dictionary.load("./dictionary_2_gensim")

In [ ]:
#создаем корпус для каждого запроса из соответствующего набора документов
files = os.listdir("./query_docs")
for idx, f in enumerate(files):
    corpus = []
    for line in open("./query_docs/" + f, "r"):
        corpus.append(dictionary.doc2bow(((line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3 else "") ).split(" ")))

In [6]:
class MyCorpus(object):
    def __iter__(self):
        files = os.listdir("./docs/")
        for idx, f in enumerate(files):
            sys.stderr.write("\rPrepare %d / %d file" % (idx, len(files)))
            for line in open("./docs/" + f):
                yield dictionary.doc2bow(((line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3 else "") ).split(" "))
                
    def get_average_len(self):
        files = os.listdir("./docs/")
        cnt_doc = 0
        sum_len = 0
        docs_length = {}
        for idx, f in enumerate(files):
#             if idx < 393:
#                 continue
            for line in open("./docs/" + f):
                cnt_doc += 1
                x = len(((line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3 else "") ).split(" "))
                sum_len += x
                docs_length[int(line.strip().split("\t")[0])] = x
            sys.stderr.write("\rPrepare %d / %d file, cnt_doc: %d, current avg_len: %f" % \
                             (idx, len(files), cnt_doc, sum_len / float(cnt_doc)))
        return (sum_len / float(cnt_doc)), docs_length

#все, что с постфиксом 2 - для биграмм
class MyCorpus_2(object):
     def __iter__(self):
        files = os.listdir("./docs/")
        for idx, f in enumerate(files):
            sys.stderr.write("\rPrepare %d / %d file" % (idx, len(files)))
            for line in open("./docs/" + f):
                yield dictionary_2.doc2bow(get_bigram(line))

In [7]:
avg_len, docs_length = MyCorpus().get_average_len()

Prepare 394 / 395 file, cnt_doc: 394955, current avg_len: 10105.858652

In [234]:
pickle.dump(docs_length, open("docs_length", "w"))

In [8]:
print len(docs_length), docs_length.keys()[:10]

394955 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [39]:
corpus = MyCorpus()
corpora.MmCorpus.serialize('corpus_gensim', corpus)

Prepare 394 / 395 file

In [44]:
corpus = MyCorpus()
tfidf = models.TfidfModel(corpus)

Prepare 394 / 395 file

In [21]:
corpus_2 = MyCorpus_2()
tfidf_2 = models.TfidfModel(corpus_2)

Prepare 394 / 395 file

In [22]:
tfidf_2.save("./tfidf_2_gensim")

In [6]:
tfidf.save("./tfidf_gensim")

NameError: name 'tfidf' is not defined

In [9]:
tfidf = models.TfidfModel.load("./tfidf_gensim")

In [10]:
tfidf_2 = models.TfidfModel.load("./tfidf_2_gensim")

In [11]:
def get_test_data():
    f = open("./test.tsv")
    test_data = {}
    for line in f:
        test_data[int(line.strip().split("\t")[0])] = line.strip().split("\t")[1][1:-1].split(",")
    return test_data

In [12]:
test_data = get_test_data()

In [13]:
print test_data[0]

['14903', '16325', '22778', '23653', '23736', '24166', '25282', '25405', '48866', '56054', '63352', '72357', '77811', '78152', '83348', '88537', '93164', '98585', '106904', '110270', '116487', '122098', '123167', '123741', '123762', '134620', '134622', '139987', '154063', '163167', '164983', '167871', '172751', '174382', '174398', '192405', '197297', '199956', '234683', '236085', '243308', '245053', '246008', '246240', '248850', '248938', '249715', '251543', '251643', '257233', '261029', '262528', '265343', '271027', '271951', '275487', '278417', '278888', '285619', '288500', '295970', '295998', '298173', '300520', '302047', '306740', '309447', '310558', '314950', '316309', '324059', '330580', '334614', '340204', '369503', '370891', '370996', '378889']


In [36]:
print dictionary.num_docs
print dictionary_2.num_docs

NameError: name 'dictionary' is not defined

In [201]:
import gc
def get_bigram_2(l):
    return [l[i] + " " + l[i + 1] for i in range(len(l) - 1)]

def get_corpus(query_id, title = False, all_text = False, bigram = False):
    if all_text:
        f = open("./query_docs/title_" + str(query_id))
        g = open("./query_docs/text_" + str(query_id))
    elif title:
        f = open("./query_docs/title_" + str(query_id))
    else:
        f = open("./query_docs/text_" + str(query_id))
    corpus = []
    doc_nums = []
    for idx, line in enumerate(f):
        if all_text:
            line_g = g.readline()
            corpus.append(dictionary.doc2bow\
                      ((line.strip().split("\t")[1] if len(line.strip().split("\t")) == 2  else "").split(" ") \
                   +(line_g.strip().split("\t")[1] if len(line_g.strip().split("\t")) == 2  else "").split(" ") ))
#         if idx == 0:
        else:
            try:
                if bigram:
                    corpus.append(dictionary_2.doc2bow\
                                  (get_bigram_2((line.strip().split("\t")[1] if len(line.strip().split("\t")) == 2 \
                                                 else "").split(" "))))
                else:
                    corpus.append(dictionary.doc2bow\
                      ((line.strip().split("\t")[1] if len(line.strip().split("\t")) == 2  else "").split(" ")))
            except:
                print "last word is bad"
                if bigram:
                    corpus.append(dictionary_2.doc2bow\
                                  (get_bigram_2((line.strip().split("\t")[1] if len(line.strip().split("\t")) == 2 \
                                                 else "").split(" ")[:-1])))
                else:
                    corpus.append(dictionary.doc2bow\
                      ((line.strip().split("\t")[1] if len(line.strip().split("\t")) == 2  else "").split(" ")[:-1]))
    
        doc_nums.append(line.strip().split("\t")[0])
    
#     if set(test_data[query_id]) != set(doc_nums):
#         print "что-то не так"
    f.close()
    if all_text:
        g.close()
#     print len(doc_nums)
    return corpus, doc_nums

def get_docs_text(query_id, title = False):
    """
    Return docs body or title for given query
    """
    if title:
        f = open("./query_docs/title_" + str(query_id))
    else:
        f = open("./query_docs/text_" + str(query_id))
        
    dict_res = {}
    for idx, line in enumerate(f):
        dict_res[int(line.strip().split("\t")[0])] = (line.strip().split("\t")[1] \
                                                        if len(line.strip().split("\t")) == 2  else "").split(" ")
    f.close()
    return dict_res

def get_query(query_id, query_filename = "test_query_norm") :
#     query_file = open(query_file, "r")
#     query_file = open("./queries_norm_expand", "r")
    query_file = open("./queries_norm.tsv", "r")
#     query_file = open(query_filename, "r")
#     print query_id
    for line in query_file:
        if line.strip().split("\t")[0] == str(query_id):
            query_file.close()
            return line.strip().split("\t")[1].decode("utf-8")
    print "don't find query"
    query_file.close()
    
def get_keywords(query_id, query_filename = "test_query_keywords") :
#     query_file = open(query_file, "r")
#     query_file = open("./queries_norm_expand", "r")
    query_file = open("./query_keywords", "r")
#     query_file = open(query_filename, "r")
#     print query_id
    for line in query_file:
        if line.strip().split("\t")[0] == str(query_id):
            query_file.close()
            if len(line.strip().split("\t")) == 2:
                return line.strip().split("\t")[1].decode("utf-8")
            else:
                return ""
    print "don't find query"
    query_file.close()
        
#создаю словарь: номер документа - score
def create_dict_from_sims(sims, docs_for_corpus):
    res = {}
    for s in sims:
        res[int(docs_for_corpus[s[0]])] = s[1]
    return res

def get_fast_corpus(query_id, title = False):
    if title:
        f = open("./query_docs_vec/title" + str(query_id))
    else:
        f = open("./query_docs_vec/text" + str(query_id))
    corpus = pickle.load(f)
    
    f.close()
    return corpus, test_data[query_id]

def get_tfidf(query_id, coef_title = 15., bigram = False, join_title = True):
    """
    Возвращает ранк tf-idf для униграмм и биграмм
    Параметры:
        query_id - номер запроса
        coef_title - коэффицент, с которым учитывается tfidf заголовка, если они объединяются
        bigram - подсчет tfidf по биграммам
        join_title - суммировать ли ранк заголовка и текста документа
        
    Возращаемые значения:
        if join_title == True:
            словарь с общим ранком для каждого документа для этого запроса в формате {номер документа : ранк}
        else:
            аналогично, но два словаря: для тела документа и для заголовка
    """
    if bigram:
        model_tfidf = tfidf_2
        model_dictionary = dictionary_2
        corpus_text, doc_nums_text = get_corpus(query_id, bigram = bigram)
        corpus_title, doc_nums_title = get_corpus(query_id, title = True, bigram = bigram)
        query = get_query(query_id)
        l = query.split(" ")
        query_vec = dictionary_2.doc2bow([l[i] + " " + l[i + 1] for i in range(len(l) - 1)])
    else:
        model_tfidf = tfidf
        model_dictionary = dictionary
        corpus_text, doc_nums_text = get_corpus(query_id)
        corpus_title, doc_nums_title = get_corpus(query_id, title = True)
    
        query = get_query(query_id)
        query_vec = dictionary.doc2bow(query.split(" "))
    
    index_text = similarities.Similarity('./tmp', model_tfidf[corpus_text], num_features=len(model_dictionary))
    index_title = similarities.Similarity('./tmp', model_tfidf[corpus_title], num_features=len(model_dictionary))
    vec_tfidf = model_tfidf[query_vec]

    sims_text = index_text[vec_tfidf]
    sims_title = index_title[vec_tfidf]

    sims_text = sorted(enumerate(sims_text), key=lambda item: -item[1])
    sims_title = sorted(enumerate(sims_title), key=lambda item: -item[1])
    
    dict_text = create_dict_from_sims(sims_text, doc_nums_text)
    dict_title = create_dict_from_sims(sims_title, doc_nums_title)
    
    if join_title:
        dict_res = {i:dict_text[int(i)] + coef_title * dict_title[int(i)] for i in doc_nums}
        return dict_res
    else:
        return dict_text, dict_title
    
#     if bigram:
#         pickle.dump(dict_text, open("./tfidf_2/text_" + str(query_id), "w"))
#         pickle.dump(dict_title, open("./tfidf_2/title_" + str(query_id), "w"))
#     else:
#         pickle.dump(dict_text, open("./tfidf_1/text_" + str(query_id), "w"))
#         pickle.dump(dict_title, open("./tfidf_1/title_" + str(query_id), "w"))

def get_submit_tfidf(submit_name, recompute = False):
    f = open(submit_name, "w")
    f.write("QueryId,DocID\n")
    coef_title = 5.
    for idx, query_id in enumerate(sorted(test_data.items(), key = operator.itemgetter(0), reverse = False)):
        tfidf_1 = get_tfidf(query_id[0], coef_title=5., bigram = False)
        tfidf_2 = get_tfidf(query_id[0], coef_title=5., bigram = True)
        tfidf_text_2 = pickle.load(open("./tfidf_2/text_" + str(query_id[0])))
        tfidf_title_2 = pickle.load(open("./tfidf_2/title_" + str(query_id[0])))
        tfidf_text_1 = pickle.load(open("./tfidf_1/text_" + str(query_id[0])))
        tfidf_title_1 = pickle.load(open("./tfidf_1/title_" + str(query_id[0])))
        
        tfidf_1 = {i:tfidf_text_1[int(i)] + coef_title * tfidf_title_1[int(i)] for i in tfidf_text_1.keys()}
        tfidf_2 = {i:tfidf_text_2[int(i)] + coef_title * tfidf_title_2[int(i)] for i in tfidf_text_2.keys()}
        
        coef_bigram = 1.0
        
        tfidf = {i:tfidf_1[i] + coef_bigram * tfidf_2[i] for i in tfidf_1.keys()}
        
        print get_query(query_id[0]), idx
        for doc in sorted(tfidf.items(), key=operator.itemgetter(1), reverse=True)[:5]:
#             print "\t score:", doc[1],"1:", tfidf_1[doc[0]], "2:", tfidf_2[doc[0]], \
#                             "\t", get_document(str(doc[0]))[6:90]
            print "1:", tfidf_1[doc[0]], "2:", tfidf_2[doc[0]], \
                            "\t", get_document(str(doc[0]))[0:90]
            f.write(str(query_id[0]) + "," + str(doc[0]) + "\n")
    f.close()
    
def first_ranking(submit_name):
    f = open(submit_name, "w")
    ranking_result = {}
    first_rank = {}
    f.write("QueryId,DocID\n")
    coef_title_1 = 5.
    coef_title_2 = 5.
    for idx, query_id in enumerate(sorted(test_data.items(), key = operator.itemgetter(0), reverse = False)):
#         tfidf_1 = get_tfidf(query_id[0], coef_title=5., bigram = False)
#         tfidf_2 = get_tfidf(query_id[0], coef_title=5., bigram = True)
        tfidf_text_2 = pickle.load(open("./tfidf_2/text_" + str(query_id[0])))
        tfidf_title_2 = pickle.load(open("./tfidf_2/title_" + str(query_id[0])))
        tfidf_text_1 = pickle.load(open("./tfidf_1/text_" + str(query_id[0])))
        tfidf_title_1 = pickle.load(open("./tfidf_1/title_" + str(query_id[0])))
        
        tfidf_1 = {i:tfidf_text_1[int(i)] + coef_title_1 * tfidf_title_1[int(i)] for i in tfidf_text_1.keys()}
        tfidf_2 = {i:tfidf_text_2[int(i)] + coef_title_2 * tfidf_title_2[int(i)] for i in tfidf_text_2.keys()}
        
        coef_bigram = 1.0
        coef_length = 0.7
        coef_avg = 0.2
        
        tfidf = {i:tfidf_1[i] + coef_bigram * tfidf_2[i] for i in tfidf_1.keys()}
        
        tfidf_norm = {i: tfidf[i] / float(max(tfidf.values())) for i in tfidf.keys()}
        
        res_ranking ={i: tfidf_norm[i] + 0.0 * click_rank[query_id[0]][i] \
                      + coef_length * \
                      (1 - (abs(coef_avg * avg_len - float(docs_length[i])) / float(avg_len))) \
                      for i in tfidf_norm.keys()}
        
        print get_query(query_id[0]), idx
#         ranking_result[query_id[0]] = []
#         first_rank[query_id[0]] = {}
      
#         top_docs =[x[0] for x in sorted(tfidf_norm.items(), key=operator.itemgetter(1), reverse=True)[:15]]
#         print f
        count = 0
        for doc in sorted(res_ranking.items(), key=operator.itemgetter(1), reverse=True):
            if count > 4:
                break
#             if doc[0] in top_docs:
#             if count < 2:
#                 print "1:", tfidf_norm[doc[0]], "len:", docs_length[doc[0]],\
#                     "coef_len:", coef_length * \
#                           (1 - (abs(coef_avg * avg_len - float(docs_length[doc[0]])) / float(avg_len))),\
#                                 "\t", get_document(str(doc[0]))[0:80]
            count += 1
#             ranking_result[query_id[0]].append(doc[0])
#             first_rank[query_id[0]][doc[0]] = doc[1]
            f.write(str(query_id[0]) + "," + str(doc[0]) + "\n")
    f.close()
#     return ranking_result, first_rank

def second_ranking(ranking_result, first_rank):
    for idx, query_id in enumerate(sorted(test_data.items(), key = operator.itemgetter(0), reverse = False)):
        print get_query(query_id[0]), idx
        
        for doc in ranking_result:
            for doc in sorted(click_rank[query_id[0]].items(), key=operator.itemgetter(1), reverse=True):
    #             print doc[0]
                if doc[0] in ranking_result[query_id[0]]:
                     print "tfidf:", first_rank[query_id[0]][doc[0]] , "click_rank:", doc[1], \
                                "\t", get_document(str(doc[0]))[0:90]
                        
def ranking():
    r, f = first_ranking()
#     second_ranking(r, f)
    
# "./test_click_data/res"\"./test_click_data/test_all_info"
def get_click_rank(res_filename = "./rec_click/res_proba", info_filename = "./rec_click/test_rec_click_info"):
    #номер запроса: {номер докуента: вероятность клика}
    click_rank = {}
    res_file = open(res_filename, "r")
    info_file = open(info_filename, "r")
    
    for line_1 in res_file:
        line_2 = info_file.readline()
        if int(line_2.strip().split(" ")[0]) not in click_rank:
            click_rank[int(line_2.strip().split(" ")[0])] = {}
        click_rank[int(line_2.strip().split(" ")[0])][int(line_2.strip().split(" ")[1])] = float(line_1.strip())
    return click_rank

In [137]:
first_ranking("./tfidf_len_c07s02_5_2")

как называться форма привести на рисунок фио какой организация представлять должность дата и способ направление приглашение подтверждение получение приглашение дата согласие принять участие в работа да нет предполагать ли выступить тема выступление нуждаться ли в гостиница нуждаться ли в билет на обратный путь самолёт поезд и так п прибыть на заседание киселёв мария владимирович ао восток главный бухгалтер 12 04 14 письмо приглашение 14 04 14 0
100 к 1 что нужно взять в 1
livestreet легко ли создать сайт новичок 2
mp3 можно переформатировать в wav 3
как востановить удалённый почтовый ящик 4
как поменять номер в привата 24 5
как шить двойной игла 6
когда выйти мулан 3 7
steam здравствуйте nexon ваш запрос на возврат средство быть получить мы рассматривать ваш запрос на возврат средство и постараться ответить как можно скорее the forest купить 13 янв pst запрос на возврат 233 83 pуб 8
wwm helena hp 200 мл black hu b можно использовать чернила для картридж hp 652 мфу hp deskjet 2135 дробн

In [23]:
click_rank = get_click_rank()

In [54]:
print sorted(click_rank[90].keys())

[205, 1401, 26921, 29996, 40307, 40408, 40414, 41790, 47785, 47925, 53863, 64427, 76645, 77538, 78301, 81853, 81931, 83193, 87443, 89521, 91778, 91780, 91846, 93510, 99426, 100313, 102962, 110988, 111584, 111597, 111653, 111670, 111671, 113381, 113981, 128432, 128983, 129481, 129739, 129782, 132808, 148413, 155523, 157377, 160140, 162992, 163234, 167900, 177233, 180551, 192353, 199504, 202948, 202950, 210079, 210932, 210996, 211471, 213415, 221397, 224355, 236073, 242873, 246364, 250113, 252389, 252490, 252597, 254954, 255465, 255662, 255750, 256424, 256435, 256811, 258218, 262579, 263524, 265064, 265089, 268974, 272062, 275332, 278842, 282036, 285182, 286774, 286775, 287191, 290560, 291648, 298555, 302144, 303133, 308013, 318760, 328882, 332251, 333412, 334856, 350906, 356037, 361781, 361794, 369186, 376094, 380372, 390172]


In [56]:
for idx, query_id in enumerate(sorted(test_data.items(), key = operator.itemgetter(0), reverse = False)):
    print get_query(query_id[0]), idx, query_id[0]
    for doc in sorted(click_rank[query_id[0]].items(), key=operator.itemgetter(1), reverse=True)[:5]:
#             print "\t score:", doc[1],"1:", tfidf_1[doc[0]], "2:", tfidf_2[doc[0]], \
#                             "\t", get_document(str(doc[0]))[6:90]
#         print doc[0], query_id[0]
        print "1:", click_rank[query_id[0]][doc[0]], \
                        "\t", get_document(str(doc[0]))[0:90]
        
# get_submit("tfidf_5_bigram_5_b1_5")

1 как называться форма привести на рисунок фио какой организация представлять должность дата и способ направление приглашение подтверждение получение приглашение дата согласие принять участие в работа да нет предполагать ли выступить тема выступление нуждаться ли в гостиница нуждаться ли в билет на обратный путь самолёт поезд и так п прибыть на заседание 1 2 3 4 5 6 7 8 9 киселёв мария владимирович ао восток главный бухгалтер 12 04 14 письмо приглашение 14 04 14 да нет нет нет да 0 0
1: 0.0247317 	22778	новость	соглашение библиотека 35788727 27552 мы в face
1: 0.0243757 	271027	л а юзефович самодержец пустынь	л а юзефович �
1: 0.0236726 	306740	против сомина часть 2	вернуться на сайт главн�
1: 0.0229644 	23653	новость	библиотека 35761384 209 мы в facebook всего нов�
1: 0.0226126 	370996	лобсанга рампа третий глаз	а б в г д е ж з и й к 
100 к 1 что нужно взять в 1 4
1: 0.0223261 	17997	поетичні майстерний самвидав	голосуванний то
1: 0.0213651 	363941	повесть для свой камертон	сетевой ли

TypeError: 'NoneType' object has no attribute '__getitem__'

In [114]:
# get_query(14)
tfidf_text = get_tfidf(990)

In [119]:
print get_query(990)
for f in sorted(tfidf_text.items(), key=operator.itemgetter(1), reverse=True)[:5]:
    print "doc_id:", f[0], "\t score:", f[1]
    print get_document(str(f[0]))[:100]

как в детский сад выдавать хлеб
doc_id: 258836 	 score: 4.3805398196
258836	питание в детский сад норма и правило	главный друг�
doc_id: 374404 	 score: 2.12472718209
374404	рецепт котлета как в детский сад секрет выбор ингр�
doc_id: 366812 	 score: 2.04824786261
366812	рыбный запеканка как в детский сад 3 рецепт	красота
doc_id: 284346 	 score: 1.9404661525
284346	оладья как в детский сад кулинарный рецепт	получат
doc_id: 267170 	 score: 1.86009597778
267170	детский монтессорить сад маленькая мы	образование


In [20]:
class ClickModel:
    def __init__(self):
        self.click_dict = {}
    
    #file_name - имя файла с кликовой информацией
    #номер_запроса - серп - номера кликнутых документов
    def fit(self, file_name):
        data_file = open(file_name, "r")
        for idx_line, line in enumerate(data_file):
            query_id = int(line.strip().split("\t")[0])
            serp = line.strip().split("\t")[1][1:-1].split(",")
            serp_click = line.strip().split("\t")[2][1:-1].split(",")
            self.click_dict[query_id] = {int(doc_id): 1 if doc_id in serp and doc_id in serp_click else 0 for doc_id in serp}
        data_file.close()
        return self
    
def get_doc_vector(doc_id):
    if int(doc_id) == 0:
        file_name = "./docs/docs_norm_1.tsv"
    else:
        file_name = "./docs/docs_norm_" + str((int(doc_id) - 1) / 1000 + 1) + ".tsv"
#     file_name = "./docs/docs_norm_" + str((int(doc_id) - 1) / 1000 + 1) + ".tsv"
    docs_file = open(file_name, "r")
    ok = 0
    for doc_line in docs_file:
        if doc_line.strip().split("\t")[0] == doc_id:
            if len(doc_line.strip().split("\t")) == 1:
                doc_vec = dictionary.doc2bow([])
            elif len(doc_line.strip().split("\t")) == 2:
                doc_vec = dictionary.doc2bow(doc_line.strip().split("\t")[1].split(" "))
            else:
                doc_vec = dictionary.doc2bow(doc_line.strip().split("\t")[1].split(" ") \
                                             + doc_line.strip().split("\t")[2].split(" "))
            ok = 1
            
    if ok != 1:
        print "doc not found", doc
    docs_file.close()
    return doc_vec

#формируем датасет для предсказания клика и сохраняем в libsvm формат:
#конкатенируем векторное представление запроса и документа
def create_click_dataset():
    y = []
    x = []
    data_file = open("./clicks.train.tsv", "r")
    res_file = open("clicks_dataset.libfm", "w")
    for idx_line, line in enumerate(data_file):
#         if idx_line > 0:
#             break
        query_id = int(line.strip().split("\t")[0])
        serp = line.strip().split("\t")[1][1:-1].split(",")
        serp_click = line.strip().split("\t")[2][1:-1].split(",")

        query = get_query(query_id)
        query_vec = dictionary.doc2bow(query.split(" "))
        for doc_idx, doc_id in enumerate(serp):
            doc_vec = get_doc_vector(doc_id)
            if doc_id in serp_click:
                res_file.write("1 ")
            else:
                res_file.write("0 ")
                
            for t in query_vec:
                res_file.write(str(t[0]) + ":" + str(t[1]) + " ")
            for t in doc_vec:
                res_file.write(str(len(dictionary) + t[0]) + ":" + str(t[1]) + " ")
            res_file.write("\n")
            
            sys.stderr.write("\rPrerare %d / %d document of %d / %d line" % (doc_idx, len(serp), idx_line, 3309))
    data_file.close()
    res_file.close()
    

#формируем анологичный датасет для тестовых данных. Для каждого запроса - свой
#конкатенируем запрос и документ (векторные представления)
def get_test_click_dataset():
    #из-за ограниченности оперативки делим тестовые данные на 5 частей
    cnt_file = 5
    cnt_query_in_file = len(test_data.keys()) / cnt_file + 1
    first_query_in_file = 0
    for f_num in range(0, 5):
        f_data = open("./test_click_data/test_click_" + str(f_num), "w")
        f_info = open("./test_click_data/test_click_"   + str(f_num) + "_info", "w")
        
        for query_idx, query_id in enumerate(sorted(test_data.keys())\
                                       [first_query_in_file : first_query_in_file + cnt_query_in_file]):
            query = get_query(query_id)
            query_vec = dictionary.doc2bow(query.split(" "))
                
            docs_id = test_data[query_id]
            for doc_idx, doc_id in enumerate(docs_id):
                doc_vec = get_doc_vector(doc_id)
                
                f_data.write("0 ")
                for t in query_vec:
                    f_data.write(str(t[0]) + ":" + str(t[1]) + " ")
                for t in doc_vec:
                    f_data.write(str(len(dictionary) + t[0]) + ":" + str(t[1]) + " ")
                f_data.write("\n")
            
                f_info.write(str(query_id) + " " + str(doc_id) + "\n")
                
                sys.stderr.write("\rPrerare %d / %d document for %d / %d query, cur_file: %d" % \
                                 (doc_idx, len(docs_id), query_idx, len(test_data.keys()), f_num))
        
        first_query_in_file = first_query_in_file + cnt_query_in_file
        f_data.close()
        f_info.close()
        

#предыдущее нихера не заработало, попробую по другому: как в рекомендательных системах
#id_запроса - id_документа
def create_rec_click_dataset(click_filename = "clicks.train.tsv"):
    click_file = open(click_filename, "r")
    res_file = open("click_dataset_rec", "w")
    for line in click_file:
        query_id = int(line.strip().split("\t")[0])
        serp = line.strip().split("\t")[1][1:-1].split(",")
        serp_click = line.strip().split("\t")[2][1:-1].split(",")
        
        for doc_id in serp:
            if doc_id in serp_click:
                res_file.write("1 ")
            else:
                res_file.write("0 ")
            res_file.write(str(query_id) + " " + str(doc_id) + "\n")
                
    res_file.close()
    click_file.close()
    
def get_test_rec_click_dataset():
    f_data = open("./rec_click/test_rec_click", "w")
    f_info = open("./rec_click/test_rec_click_info", "w")

    for query_idx, query_id in enumerate(sorted(test_data.keys())):
        docs_id = test_data[query_id]
        for doc_id in docs_id:
            f_data.write("0 ")
            f_data.write(str(query_id) + " " + str(doc_id) + "\n")

            f_info.write(str(query_id) + " " + str(doc_id) + "\n")

    f_data.close()
    f_info.close()

In [21]:
create_rec_click_dataset()

In [51]:
get_test_rec_click_dataset()

In [35]:
get_test_click_dataset()

Prerare 106 / 107 document for 159 / 816 query, cur_file: 4

In [27]:
click_model = ClickModel().fit("./clicks.train.tsv")
ctr_dict = click_model.click_dict

In [31]:
for q in test_data.keys():
    if q in ctr_dict:
        print q

In [33]:
print ctr_dict.keys()
print test_data.keys()

[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 44, 45, 48, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 64, 65, 66, 67, 68, 69, 70, 71, 72, 75, 76, 77, 78, 79, 80, 81, 82, 85, 86, 88, 89, 91, 92, 94, 96, 98, 100, 101, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 123, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 163, 165, 167, 169, 171, 172, 174, 175, 176, 177, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 192, 193, 194, 195, 201, 204, 205, 206, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 227, 228, 229, 231, 232, 234, 235, 236, 237, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 263, 264, 265, 266, 268, 269, 270, 271, 272, 273, 274, 2

In [ ]:
create_click_dataset()

In [43]:
validate_line = {np.random.randint(CNT_CLICK_INFO): 0 for i in range(int(CNT_CLICK_INFO * 0.2))}

def train_val_separate(file_name, validate_line):
    train_file = open(file_name + "_train", "w")
    val_file = open(file_name + "_val", "w")
    with open(file_name, "r") as f:
        for idx, line in enumerate(f):
            #костыль, потому что мне в оперативку не влазит
            if idx >= (CNT_CLICK_INFO * 0.1):
                break
            if idx in validate_line:
                val_file.write(line)
            else:
                train_file.write(line)
    train_file.close()
    val_file.close()

In [44]:
train_val_separate("./rec_click/click_dataset_rec.libfm", validate_line)

In [9]:
#подпилим Word2Vec - уже обученный мной в рамках курсовой
with open('word_vectors', "r") as f:
    WordVectors = pickle.load(f)

In [10]:
x = WordVectors.most_similar(u'страна')
for w in x:
    print w[0], w[1]

print
x = WordVectors.most_similar(u'сша')
for w in x:
    print w[0], w[1]

государство 0.709697008133
регион 0.663186252117
континент 0.594460129738
украина 0.562653005123
азербайджан 0.551063299179
россия 0.550385415554
латвия 0.533351778984
евросоюз 0.527279913425
турция 0.513354420662
америка 0.511987745762

американский 0.726210415363
вашингтон 0.697807908058
американец 0.636369884014
турция 0.618627905846
пентагон 0.6002997756
иран 0.597927212715
америка 0.592969357967
китай 0.591142654419
сирия 0.590399384499
филиппина 0.585408568382


In [70]:
#пробуем дополнять запросы при помощи Word2Vес - брать N топовых слов по схожести с данным (но не для стоп-слов)
def expand_query(query, word_vectors, top_N, border = 0.9):
    words = query.strip().split(" ")
    new_query = query
    for word in words:
        if word.decode("utf-8") not in stop_words and word.decode("utf-8") in WordVectors:
#             print word,
            sim_words = WordVectors.most_similar(word.decode("utf-8"))
            for i in range((min(len(sim_words), top_N))):
#                 print sim_words[i][0]
                if sim_words[i][1] >= border:
#                     print 1, 
                    new_query += " " + sim_words[i][0].encode("utf-8")
    return new_query

In [71]:
print expand_query("где взять кредит без справка о доход", WordVectors, 1)

где взять кредит без справка о доход


In [72]:
with open("queries_norm.tsv", "r") as q_file:
    new_q_file = open("queries_norm_expand", "w")
    for idx, line in enumerate(q_file):
        sys.stderr.write("\r %d" % (idx))
        new_q_file.write(expand_query(line.strip(), WordVectors, top_N= 1) + '\n')
    new_q_file.close()

 4124

In [123]:
print u'страна' in WordVectors

True


In [13]:
for idx, line in enumerate(open("./docs/docs_norm_1.tsv", "r")):
    if idx >= 1:
        break
    print len(line.split(" "))
    print len(( (line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3  else "") ).split(" "))
    bprint(( (line.strip().split("\t")[1] if len(line.split("\t")) >= 2  else "") + " "\
                                     + (line.strip().split("\t")[2] if len(line.split("\t")) == 3  else "") ).split(" "))

456
457
smartjs
academy
архитектура
js
приложение
харьков
о
курс
для
кто
тема
цена
тренер
вопрос
архитектура
js
приложение
место
нет
возможный
запись
в
лист
ожидание
в
течение
последний
год
я
размышлять
о
курс
который
бы
заполнять
пробел
между
базовый
познание
javascript
и
конкретный
фреймворка
курс
по
фреймворка
напоминать
я
игра
на
балалайка
делать
раз
делать
два
помнить
о
проблема
получать
результат
как
итог
на
выход
человек
уметь
писать
на
конкретный
фреймворка
а
не
решать
задача
с
появление
reactjs
курс
стать
полезный
и
в
коммерческий
разработка
reactjs
не
делать
предположение
о
архитектура
приложение
а
значит
для
эффективный
разработка
необходимый
понимание
как
правильно
структурировать
проект
и
ура
ура
это
чистый
js
отдельный
привет
angularjs
для
кто
вы
интересоваться
современный
мир
js
разработка
и
универсальный
подход
к
разработка
приложение
вы
писать
на
какой
то
фреймворка
и
чувствовать
как
он
ограничать
ваш
возможность
вы
слышать
о
модный
reactjs
и
желать
научиться
программи

# ЩА БУДЕТ МАШИННОЕ ОБУЧЕНИЕ АТТЕНШН

In [203]:
class TextFeatureGenerator():
    """
    Generate all text features for all documents for given query
    query in unicode
    text in utf-8
    """
    def __init__(self, query_id):
        self.docs_text = get_docs_text(query_id)
        self.docs_title = get_docs_text(query_id, title=True)
        self.query_id = query_id
        self.query_text = get_query(query_id).split(" ")
        self.keywords_text = get_keywords(query_id).split(" ")
        self.is_geo_query, self.geo = self.search_city(self.query_text)
#         if self.is_geo_query:
#             print "is geo query"
        
    def search_city(self, text):
#         print text
        for word in text:
            if word in city_list:
                return True, word
        return False, None
    
    def search_city_in_text(self, text, city):
        text_str = " ".join(text).decode("utf-8").split(" ")
        if city in text_str:
            return 1.0
        else:
            return 0.0

    #TODO: подумать над значением по умолчанию
    def find_first_meeting(self, query, text, default_value = sys.maxint):
#         query_decode = " ".join(query).decode("utf-8").split(" ")
        fm = default_value
        for idx, word in enumerate(text):
            if word.decode("utf-8") in query:
                return idx
        return fm
    
    def find_min_distance(self, query, text):
        try:
            words_indices = [[i for i, x in enumerate(text) if x.decode("utf-8") == query_word] for query_word in query]
        except:
            print "WARNING: can't decode utf-8"
            words_indices = [[i for i, x in enumerate(text[:-1]) if x == query_word] for query_word in query]
        min_distance = sys.maxint
#         print len(words_indices)
        for idx_1, w_ind_1 in enumerate(words_indices):
            for idx_2, w_ind_2 in enumerate(words_indices[idx_1:]):
                if idx_1 == idx_2:
                    continue
                for i1 in w_ind_1:
                    for i2 in w_ind_2:
                        if abs(i1 - i2) < min_distance:
                            min_distance = abs(i1 - i2)
                            if min_distance == 1:
                                return min_distance
        return min_distance
    
    def is_query_in_text(self, query, text):
        query_str = " ".join(query)
        try:
            text_str = " ".join(text).decode("utf-8")
        except:
            print "WARNING: can't decode utf-8"
            text_str = (" ".join(text[:-1])).decode("utf-8")
        if text_str.find(query_str) != -1:
            return 1
        else:
            return 0
        
    def part_query_in_text(self, query, text):
        try:
            text_decode = " ".join(text).decode("utf-8").split(" ")
        except:
            print "WARNING: can't decode utf-8"
            text_decode = " ".join(text[:-1]).decode("utf-8").split(" ")
        query_len = len(query)
        cnt = 0
        for word in query:
            if word in text_decode:
                cnt += 1
        return float(cnt) / query_len
        
    def get_text_features(self):
#         print self.query_id, "\t", " ".join(self.query_text)
#         print "TFIDF"
        
        tfidf_text, tfidf_title = get_tfidf(self.query_id, join_title=False)
        tfidf_text_2, tfidf_title_2 = get_tfidf(self.query_id, bigram=True, join_title=False)
        
        text_features = {}
        
        for doc_num, doc_id in enumerate(self.docs_text.keys()):
#             if doc_num > 0:
#                 break
#             print "FIRST MEETING"
            if (doc_id not in self.docs_text) or (doc_id not in self.docs_title):
                print "can't find document:", doc_id 
                continue
            fm_text = self.find_first_meeting(self.query_text, self.docs_text[doc_id])
            fm_title = self.find_first_meeting(self.query_text, self.docs_title[doc_id])
#             print "LENGTH"
            length_text, length_title = len(self.docs_text[doc_id]), len(self.docs_title[doc_id])
#             print self.docs_title[doc_id]
#             print "MIN_DISTANCE"
            min_distance_text = self.find_min_distance(self.query_text, self.docs_text[doc_id])
            min_distance_title = self.find_min_distance(self.query_text, self.docs_title[doc_id])
#             print "Q IN FILE"
            is_q_in_text = self.is_query_in_text(self.query_text, self.docs_text[doc_id])
            is_q_in_title = self.is_query_in_text(self.query_text, self.docs_title[doc_id])
#             print "PART IN Q"
            part_q_in_text = self.part_query_in_text(self.query_text, self.docs_text[doc_id])
            part_q_in_title = self.part_query_in_text(self.query_text, self.docs_title[doc_id])
        
            is_all_words_in_text = 1 if part_q_in_text == 1.0 else 0
            is_all_words_in_title = 1 if part_q_in_title == 1.0 else 0
            
            if self.is_geo_query:
                is_geo_in_text = self.search_city_in_text(self.docs_text[doc_id], self.geo)
                is_geo_in_title = self.search_city_in_text(self.docs_title[doc_id], self.geo)
            else:
                is_geo_in_text = 0.0
                is_geo_in_title = 0.0
                
            part_key_in_text = self.part_query_in_text(self.keywords_text, self.docs_text[doc_id])
            part_key_in_title = self.part_query_in_text(self.keywords_text, self.docs_title[doc_id])
        
            is_all_keywords_in_text = 1 if part_key_in_text == 1.0 else 0
            is_all_keywords_in_title = 1 if part_key_in_title == 1.0 else 0
            
            text_features[doc_id] = [tfidf_text[doc_id], tfidf_title[doc_id], 
                                     tfidf_text_2[doc_id], tfidf_title_2[doc_id],
                                     length_text, length_title,
                                     fm_text, fm_title,
                                     min_distance_text, min_distance_title,
                                     is_q_in_text, is_q_in_title,
                                     part_q_in_text, part_q_in_title, 
                                     is_all_words_in_text, is_all_words_in_title, 
                                     is_geo_in_text, is_geo_in_title,
                                     part_key_in_text, part_key_in_title,
                                     is_all_keywords_in_text, is_all_keywords_in_title]
#             print text_features
#             print get_document(str(doc_id))
        
        return text_features
        

def create_train_text_factors_dataset(click_filename = "clicks.train.tsv"):
    click_file = open(click_filename, "r")
    res_file = open("./text_click_factors/text_click_dataset", "w")
#     last column - target
    train_click_data = []
    for line_idx, line in enumerate(click_file):
        query_id = int(line.strip().split("\t")[0])
        serp = line.strip().split("\t")[1][1:-1].split(",")
        serp_click = line.strip().split("\t")[2][1:-1].split(",")
        
        text_factors = TextFeatureGenerator(query_id).get_text_features()
        
        for doc_id in text_factors.keys():
            
            if str(doc_id) in serp_click:
                target = 1
            else:
                target = 0
                
            train_click_data.append(text_factors[doc_id] + [target])
            res_file.write(",".join([str(x) for x in (text_factors[doc_id] + [target])]) + "\n")
            res_file.flush()
            sys.stderr.write("\rPrepare %d / %d query, %d point in dataset" %\
                             (line_idx, 3309, len(train_click_data)))
            
#             print train_click_data
    
    return np.asarray(train_click_data)

In [ ]:
train_data = create_train_text_factors_dataset()

Prepare 0 / 3309 query, 85 point in dataset

In [351]:
train_data.dump("./text_click_factors/text_click_train_data")

In [179]:
train_data = np.load("./text_click_factors/text_click_train_data")

In [180]:
print train_data.shape

(319956, 15)


In [183]:
X = train_data.copy()[:, :-1]
Y = train_data.copy()[:, -1]
print X.shape
print Y.shape

(319956, 14)
(319956,)


In [329]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [181]:
# gbc = GradientBoostingRegressor()
rf = RandomForestRegressor()

In [182]:
parametrs_grid = {'n_estimators': [600, 1000, 1300, 1500], \
                  'max_depth':[2, 3, 4]}

In [101]:
print set(Y)

set([0.0, 1.0])


In [184]:
grid_model = GridSearchCV(rf, parametrs_grid, cv = 5, verbose = 3)
grid_model.fit(X, Y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] n_estimators=600, max_depth=2 ...................................
[CV] .......... n_estimators=600, max_depth=2, score=0.037601 -   1.1s
[CV] n_estimators=600, max_depth=2 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.6min remaining:    0.0s


[CV] .......... n_estimators=600, max_depth=2, score=0.023670 -   1.1s
[CV] n_estimators=600, max_depth=2 ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.1min remaining:    0.0s


[CV] .......... n_estimators=600, max_depth=2, score=0.023103 -   1.4s
[CV] n_estimators=600, max_depth=2 ...................................
[CV] .......... n_estimators=600, max_depth=2, score=0.034850 -   1.1s
[CV] n_estimators=600, max_depth=2 ...................................
[CV] .......... n_estimators=600, max_depth=2, score=0.027510 -   1.1s
[CV] n_estimators=1000, max_depth=2 ..................................
[CV] ......... n_estimators=1000, max_depth=2, score=0.037616 -   1.8s
[CV] n_estimators=1000, max_depth=2 ..................................
[CV] ......... n_estimators=1000, max_depth=2, score=0.023623 -   2.3s
[CV] n_estimators=1000, max_depth=2 ..................................
[CV] ......... n_estimators=1000, max_depth=2, score=0.023207 -   2.0s
[CV] n_estimators=1000, max_depth=2 ..................................
[CV] ......... n_estimators=1000, max_depth=2, score=0.034837 -   1.9s
[CV] n_estimators=1000, max_depth=2 ..................................


KeyboardInterrupt: 

In [20]:
gbc = GradientBoostingRegressor(n_estimators=1600, max_depth=2, verbose=1).fit(X, Y)

      Iter       Train Loss   Remaining Time 
         1           0.0154           10.43m
         2           0.0153           10.01m
         3           0.0153            9.97m
         4           0.0152            9.84m
         5           0.0152            9.82m
         6           0.0152            9.74m
         7           0.0151            9.73m
         8           0.0151            9.71m
         9           0.0151            9.67m
        10           0.0151            9.65m
        20           0.0150            9.20m
        30           0.0150            8.90m
        40           0.0150            8.72m
        50           0.0149            8.65m
        60           0.0149            8.58m
        70           0.0149            8.47m
        80           0.0149            8.39m
        90           0.0149            8.29m
       100           0.0149            8.20m
       200           0.0149            7.42m
       300           0.0148            6.82m
       40

In [185]:
rf = GradientBoostingRegressor(n_estimators=1000, max_depth=2, verbose=1).fit(X, Y)

      Iter       Train Loss   Remaining Time 
         1           0.0154            6.61m
         2           0.0153            6.70m
         3           0.0153            6.76m
         4           0.0152            6.68m
         5           0.0152            6.64m
         6           0.0152            6.70m
         7           0.0151            6.96m
         8           0.0151            7.14m
         9           0.0151            7.02m
        10           0.0151            6.93m
        20           0.0150            6.35m
        30           0.0150            6.33m
        40           0.0150            6.89m
        50           0.0149            6.85m
        60           0.0149            6.86m
        70           0.0149            6.89m
        80           0.0149            6.87m
        90           0.0149            6.81m
       100           0.0149            6.78m
       200           0.0149            6.09m
       300           0.0148            5.28m
       40

In [21]:
print gbc.feature_importances_
# [tfidf_text[doc_id], tfidf_title[doc_id], tfidf_text_2[doc_id], tfidf_title_2[doc_id],
# length_text, length_title, fm_text, fm_title,
# min_distance_text, min_distance_title, is_q_in_text, is_q_in_title,
# part_q_in_text, part_q_in_title, is_all_words_in_text, is_all_words_in_title, 
# is_geo_in_text, is_geo_in_title, part_key_in_text, part_key_in_title,
# is_all_keywords_in_text, is_all_keywords_in_title]

[ 0.21037814  0.21101825  0.22320036  0.11248609  0.07603711  0.03541653
  0.02091097  0.00613161  0.00104691  0.00598563  0.00197722  0.00465221
  0.01957181  0.07118715]


In [150]:
text_factors_dict = {}

In [189]:
def rank_text_click_factors(query_id):
    if TextFeatureGenerator(query_id).is_geo_query:
        print "Geo query"
    if query_id in text_factors_dict:
        text_factors = text_factors_dict[query_id]
    else:
        text_factors = TextFeatureGenerator(query_id).get_text_features()
        text_factors_dict[query_id] = text_factors
        
    np_data = np.asarray([f for f in text_factors.values()])
    prepare_model = MinMaxScaler().fit(np_data)
    
    coef = [1, 5, 1, 5,\
            0., 0, 0.0, 0.0, \
            1.0, 1.0, 3.0, 7.0,\
            3.0, 7.0, 1.0, 1.0,\
           3.0, 5.0, 0.0, 0.0, \
            0.0, 0.0]
#     print set(list(np_data[:, 8].ravel()))    
    text_click_rank = {}
    for idx, doc_id in enumerate(text_factors.keys()):
        prepare_data = prepare_model.transform(np.asarray ([text_factors[doc_id]]))
        rank = 0
        for i, x in enumerate(prepare_data[0]):
            if i == 6 or 1 == 7 or i == 8:
#                 if i == 8:
#                     print x, coef[i] * (1 - x)
                rank += coef[i] * (1 - x)
            else:
                rank += coef[i] * x
#         print prepare_data
#         print rank    
#         text_click_rank[doc_id] = gbc.predict(np.asarray([text_factors[doc_id]]))
        rank += 25 * gbc.predict(np.asarray([text_factors[doc_id][:14]]))
        rank += 40 * rf.predict(np.asarray([text_factors[doc_id][:14]]))
        coef_length = 0.7
        coef_avg = 0.2
        rank += coef_length * (1 - (abs(coef_avg * avg_len - float(docs_length[doc_id])) / float(avg_len))) 
#         print coef_length * (1 - (abs(coef_avg * avg_len - float(docs_length[doc_id])) / float(avg_len))) 
        text_click_rank[doc_id] = rank
#         sys.stderr.write("\rPrepare %d / %d document" % (idx, len(text_factors)))
    return text_click_rank

def get_submit(submit_name):
    f = open(submit_name, "w")
    f.write("QueryId,DocID\n")
    log = open("log_1", "w")
    for idx, query_id in enumerate(sorted(test_data.items(), key = operator.itemgetter(0), reverse = False)):
        rank = rank_text_click_factors(query_id[0])
#         print rank
        print get_query(query_id[0]), idx
        log.write(get_query(query_id[0]).encode("utf-8") + " " + str(idx) + ' \n'.encode("utf-8"))
        for doc in sorted(rank.items(), key=operator.itemgetter(1), reverse=True)[:5]:
#             print "\t score:", doc[1],"1:", tfidf_1[doc[0]], "2:", tfidf_2[doc[0]], \
#                             "\t", get_document(str(doc[0]))[6:90]
            if get_document(str(doc[0])) is not None:
#                 pass
#                 print rank[doc[0]], "\t", get_document(str(doc[0]))[0:90]
                log.write(str(rank[doc[0]]) + "\t" + get_document(str(doc[0]))[0:90] \
                          + ' \n'.encode("utf-8"))
            f.write(str(query_id[0]) + "," + str(doc[0]) + "\n")
    f.close()
    log.close()

In [190]:
get_submit("./text_click_factors/submit_1234910_length_11_12_13_14_15_16_gbt_geo_rf")

как называться форма привести на рисунок фио какой организация представлять должность дата и способ направление приглашение подтверждение получение приглашение дата согласие принять участие в работа да нет предполагать ли выступить тема выступление нуждаться ли в гостиница нуждаться ли в билет на обратный путь самолёт поезд и так п прибыть на заседание киселёв мария владимирович ао восток главный бухгалтер 12 04 14 письмо приглашение 14 04 14 0
100 к 1 что нужно взять в 1
livestreet легко ли создать сайт новичок 2
mp3 можно переформатировать в wav 3
как востановить удалённый почтовый ящик 4
как поменять номер в привата 24 5
как шить двойной игла 6
когда выйти мулан 3 7
steam здравствуйте nexon ваш запрос на возврат средство быть получить мы рассматривать ваш запрос на возврат средство и постараться ответить как можно скорее the forest купить 13 янв pst запрос на возврат 233 83 pуб 8
wwm helena hp 200 мл black hu b можно использовать чернила для картридж hp 652 мфу hp deskjet 2135 дробн

In [61]:
text_factors = TextFeatureGenerator(577).get_text_features()

[u'\u0433\u0434\u0435', u'\u043a\u0443\u043f\u0438\u0442\u044c', u'\u043a\u0430\u0444\u0435\u043b\u044c\u043d\u044b\u0439', u'\u043f\u043b\u0438\u0442\u043a\u0430', u'\u0432', u'\u0431\u043b\u0430\u0433\u043e\u0432\u0435\u0449\u0435\u043d\u0441\u043a']
is geo query


In [68]:
print np.asarray(text_factors.values())[:, -2:]

[[ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 1.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 1.  0.]]


In [166]:
def show_difference(submit_1, submit_2):
    submit_1 = open(submit_1, "r")
    submit_2 = open(submit_2, "r")
    docs_1 = []
    docs_2 = []
    cnt_different = 0
    for idx, line_1 in enumerate(submit_1):
        line_2 = submit_2.readline()
        if idx == 0:
            continue
        cur_query_id = line_1.strip().split(",")[0]
        docs_1.append(int(line_1.strip().split(",")[1]))
        docs_2.append(int(line_2.strip().split(",")[1]))
#         print docs_1, docs_2
        if (idx) % 5 == 0:
#             print docs_1, docs_2
            dif = False
            for i, x in enumerate(docs_1):
                if docs_2[i] != x:
                    dif =  True
                    cnt_different += 1
                    break
            if dif:
                print cur_query_id,  get_query(cur_query_id)
                print "SERP_1:"
                for doc in docs_1:
                    print "\t", get_document(str(doc))[0:100]
                print "SERP_2:"
                for doc in docs_2:
                    print "\t", get_document(str(doc))[0:100]
            docs_1 = []
            docs_2 = []
    print "Count of query with different ranking: ",cnt_different 

In [191]:
show_difference("./text_click_factors/submit_1234910_length_11_12_13_14_15_16_gbt_geo", \
                "./text_click_factors/submit_1234910_length_11_12_13_14_15_16_gbt_geo_rf")

31 как шить двойной игла
SERP_1:
	45750	как шить двойной игла youtube	как шить двойной игла еле
	393969	как шить двойной игла шить сам	шить сам блог для то�
	293694	как шить двойной игла ответ здесь	категория другой
	159900	как шить двойной игла как шить двойной игла игла ун
	44651	как шить двойной игла прямой строчка машинный стро�
SERP_2:
	393969	как шить двойной игла шить сам	шить сам блог для то�
	45750	как шить двойной игла youtube	как шить двойной игла еле
	293694	как шить двойной игла ответ здесь	категория другой
	44651	как шить двойной игла прямой строчка машинный стро�
	159900	как шить двойной игла как шить двойной игла игла ун
46 wwm helena hp 200 мл black hu b можно использовать чернила для картридж hp 652 мфу hp deskjet 2135 дробный http rozetka com ua wwm hu b p2704482
SERP_1:
	391294	потребительский товар чернила wwm для hp в украина ср
	385259	страница 3 раздел расходный материал совместимый с
	385278	rozetka ua чернила wwm helena hp 200 мл cyan hu c цена купить чернил
	

In [161]:
pickle.dump(text_factors_dict, open("./text_click_factors/text_factors_dict", "w"))

In [281]:
query = ["трололо", "кекеке", "sjdgbjgasgsrg"]
text = ["воаиплв","dgsdg",  "трололо","кекеке", "xdgdfh", "sdgdfhgj", "трололо", "sjgn", "кекеке"]

In [282]:
t = TextFeatureGenerator(2)
t.get_text_features()

2 	1 положительный и 1 отрицательный мочь ли
TFIDF
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH
MIN_DISTANCE
7
7
Q IN FILE
PART IN Q
FIRST MEETING
LENGTH


{20113: [0.023856573,
  0.0,
  0.0017775961,
  0.0,
  52022,
  4,
  11,
  9223372036854775807,
  0,
  9223372036854775807,
  0,
  0,
  1.0,
  0.0],
 20257: [0.20845853,
  0.0,
  0.085343592,
  0.0,
  5962,
  8,
  68,
  9223372036854775807,
  1,
  9223372036854775807,
  0,
  0,
  1.0,
  0.0],
 23297: [0.020820325,
  0.0,
  0.00155345,
  0.0,
  62852,
  1,
  34,
  9223372036854775807,
  0,
  9223372036854775807,
  0,
  0,
  1.0,
  0.0],
 23724: [0.019299163,
  0.0,
  0.00025795668,
  0.0,
  40500,
  1,
  34,
  9223372036854775807,
  0,
  9223372036854775807,
  0,
  0,
  1.0,
  0.0],
 25500: [0.021454066,
  0.0,
  0.00056606083,
  0.0,
  64059,
  1,
  34,
  9223372036854775807,
  0,
  9223372036854775807,
  0,
  0,
  1.0,
  0.0],
 25738: [0.0079277856,
  0.0,
  0.00060524244,
  0.0,
  23535,
  4,
  34,
  9223372036854775807,
  1,
  9223372036854775807,
  0,
  0,
  0.8571428571428571,
  0.0],
 27070: [0.0097970972,
  0.0,
  0.00081728218,
  0.0,
  44059,
  4,
  26,
  9223372036854775807,
 

In [ ]:
def create_term_click_dict(click_filename = "clicks.train.tsv"):
    click_file = open(click_filename, "r")
    res_file = open("click_dataset_rec", "w")
    for line in click_file:
        query_id = int(line.strip().split("\t")[0])
        serp = line.strip().split("\t")[1][1:-1].split(",")
        serp_click = line.strip().split("\t")[2][1:-1].split(",")

In [194]:
with open("./queries_norm.tsv") as f:
    w = open("./query_keywords", "w")
    for line in f:
        q = line.strip().split("\t")[1].split(" ")
        keywords = " ".join([x  for x in q if x.decode("utf-8") not in stop_words])
        w.write(line.split("\t")[0] +"\t" +  keywords + "\n")
    w.close()

In [57]:
bprint(city_list)

Чапаевск
Артёмовский
Михайловск
Льгов
Рыльск
Ирбит
Мончегорск
Октябрьский
Углегорск
Сатка
Гвардейск
Зеленогорск
Хилок
Таштагол
Пудож
Новый Оскол
Чекалин
Надым
Новороссийск
Рославль
Горняк
Южно-Сухокумск
Буй
Нолинск
Мариинск
Грайворон
Корсаков
Уфа
Шумерля
Ханты-Мансийск
Чулым
Тында
Кохма
Суздаль
Звенигород
Сольвычегодск
Перевоз
Щигры
Куса
Сосновоборск
Чаплыгин
Бугуруслан
Михайлов
Железноводск
Кыштым
Шелехов
Омутнинск
Ершов
Пестово
Новый Уренгой
Энгельс
Георгиевск
Кунгур
Нижняя Салда
Вихоревка
Островной
Калтан
Советский
Нижние Серги
Заволжье
Кологрив
Беломорск
Лесозаводск
Уржум
Кореновск
Новая Ладога
Томари
Моршанск
Нижнеудинск
Рыбное
Фокино
Липецк
Лыткарино
Приморско-Ахтарск
Бикин
Новокузнецк
Южа
Всеволожск
Краснослободск
Еманжелинск
Щербинка
Коломна
Россошь
Ишим
Хадыженск
Алатырь
Пучеж
Сосновка
Свободный
Ряжск
Хабаровск
Прокопьевск
Демидов
Малоярославец
Донецк
Юрьевец
Таганрог
Шахты
Волжск
Ростов-на-Дону
Мценск
Асбест
Уссурийск
Зверево
Славгород
Элиста
Скопин
Няндома
Котельниково
Красн